In [ ]:
pip install -Uqq fastai

In [ ]:
import pandas as pd
from fastai.text.all import *

# Dummy data for League of Legends champions and their roles
data = {
    'description': [
        "A relentless top lane brawler who thrives in extended fights, dealing sustained damage and healing.",
        "A powerful mid lane wizard, unleashing devastating magic spells from a safe distance.",
        "A sturdy frontline warrior, soaking up damage and initiating fights with crowd control.",
        "A swift and agile marksman, excelling at ranged combat and objective control.",
        "A shadowy assassin, darting through the battlefield to eliminate high-priority targets.",
        "A supportive enchanter, shielding allies and empowering them with buffs.",
        "A master of the jungle, ambushing enemies and securing early game advantages.",
        "A powerful late-game hypercarry, scaling into an unstoppable force with items."
    ],
    'role': [
        'Fighter',
        'Mage',
        'Tank',
        'Marksman',
        'Assassin',
        'Support',
        'Jungle',
        'Marksman'
    ]
}

df = pd.DataFrame(data)
display(df.head())

,description,role
0,"A relentless top lane brawler who thrives in extended fights, dealing sustained damage and healing.",Fighter
1,"A powerful mid lane wizard, unleashing devastating magic spells from a safe distance.",Mage
2,"A sturdy frontline warrior, soaking up damage and initiating fights with crowd control.",Tank
3,"A swift and agile marksman, excelling at ranged combat and objective control.",Marksman
4,"A shadowy assassin, darting through the battlefield to eliminate high-priority targets.",Assassin


In [ ]:
# Create a tokenizer instance
tokenizer = Tokenizer(tok=WordTokenizer())

# Extract all unique roles to use as vocabulary for CategoryBlock
all_roles_vocab = df['role'].unique().tolist()

# Define the DataBlock for text classification
d_block = DataBlock(blocks=(TextBlock(tok_tfm=tokenizer, seq_len=72, is_lm=False), CategoryBlock(vocab=all_roles_vocab)),
                    get_x=ColReader('description'), # Explicitly get the 'description' column as input
                    get_y=ColReader('role'),
                    splitter=RandomSplitter(valid_pct=0.2, seed=42))

# Create DataLoaders
dls = d_block.dataloaders(df, bs=4)

# Display a batch of data to verify the data preparation
dls.show_batch()

,text,category
0,"xxbos a xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk , xxunk xxunk xxunk and xxunk .",Fighter
1,"xxbos a xxunk xxunk , xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk .",Assassin
2,"xxbos a xxunk xxunk xxunk xxunk xxunk , xxunk xxunk xxunk xxunk xxunk with xxunk .",Marksman
3,"xxbos a xxunk xxunk xxunk , xxunk xxunk xxunk and xxunk xxunk with xxunk xxunk .",Tank


In [ ]:
from fastai.text.all import *
from fastai.callback.progress import ProgressCallback

# Create a text classifier learner
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

# Explicitly remove ProgressCallback to avoid display issues in some environments
learn.remove_cb(ProgressCallback)

# Fine-tune the model for a few epochs
learn.fine_tune(4, 1e-2)

[0, 2.2323930263519287, 2.002941131591797, 0.0, '00:01']
[0, 1.6690105199813843, 2.0121281147003174, 0.0, '00:01']
[1, 2.2341244220733643, 1.9931037425994873, 0.0, '00:02']
[2, 2.1933629512786865, 1.9463647603988647, 0.0, '00:02']
[3, 2.03836727142334, 1.9522615671157837, 0.0, '00:03']


In [ ]:
# Make a prediction on new text
new_champion_description = "A heavily armored champion with a large shield, capable of protecting allies and enduring immense damage."
prediction = learn.predict(new_champion_description)

print(f"Description: {new_champion_description}")
print(f"Predicted Role: {prediction[0]}")
print(f"Prediction Confidence (probabilities): {prediction[2].numpy()}")
print(f"Classes: {learn.dls.vocab[1]}")

Description: A heavily armored champion with a large shield, capable of protecting allies and enduring immense damage.
Predicted Role: Fighter
Prediction Confidence (probabilities): [0.13940687 0.1528863  0.14839552 0.13544777 0.14580995 0.13897727
 0.13907625]
Classes: ['Assassin', 'Fighter', 'Jungle', 'Mage', 'Marksman', 'Support', 'Tank']
